In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir("/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/")

In [3]:
from os.path import splitext
from os import listdir
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import DataLoader, random_split
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import torch

In [ ]:
### MetaData1 

## Input images
# data/customdata/capstone_dataset/inputs/ppe_input_images

## Ground Truths
# data/customdata/capstone_dataset/ground_truths/depthmap_gt_images
# data/customdata/capstone_dataset/ground_truths/planar_surface_gt_images
# data/customdata/capstone_dataset/ground_truths/ppe_gt_bbox_labels

In [ ]:
### MetaData2

## Input images
# data/customdata/capstone_dataset/inputs/interior_input_images

## Ground Truths
# data/customdata/capstone_dataset/ground_truths/depthmap_gt_images
# data/customdata/capstone_dataset/ground_truths/planar_surface_gt_images

In [4]:
root = '/content/drive/My Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/'

In [5]:
datafile=[]
for path, subdirs, files in os.walk(root + 'inputs/ppe_input_images'):
  for filename in files:
    name, file_extension = os.path.splitext(filename)

    midas_out_file = name+'.png'
    input_file= root + 'inputs/ppe_input_images/'+filename
    bb_output_file = root+'ground_truths/ppe_gt_bbox_labels/'+name+'.txt'
    midas_output_file = root+'ground_truths/depthmap_gt_images/'+midas_out_file
    sp_output_file = root+'ground_truths/planar_surface_gt_images/'+name+'_segmentation_final.png'
    plane_masks_npy_output_file = root+'ground_truths/planar_surface_gt_images/'+name+'_plane_masks.npy'
    # plane_parameters_npy_output_file = root+'ground_truths/planar_surface_gt_images/'+name+'_plane_parameters.npy'

    y0 = bb_output_file if(os.path.isfile(bb_output_file)) else ''

    y1 = midas_output_file if(os.path.isfile(midas_output_file)) else ''

    y2 = sp_output_file if(os.path.isfile(sp_output_file)) else ''
    y3 = plane_masks_npy_output_file if(os.path.isfile(plane_masks_npy_output_file)) else ''
    # y4 = plane_parameters_npy_output_file if(os.path.isfile(plane_parameters_npy_output_file)) else '' 
    
    datafile.append([input_file,y0,y1,y2,y3])

In [8]:
column = ['Input','BBoutput','MidasOutput','SurfacePlane','SurfaceCoordinates']
dataset_df = pd.DataFrame(datafile,columns=column)

In [9]:
!ls /content/drive/My\ Drive/computer_vision/capstone_project/yolo_v3/data/customdata/capstone_dataset/ground_truths/planar_surface_gt_images | wc -l

42272


In [10]:
dataset_df.head()

,Input,BBoutput,MidasOutput,SurfacePlane,SurfaceCoordinates
0,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,,,
1,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,,
2,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,,
3,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,,
4,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,,


In [11]:
for col in dataset_df.columns:
  print(f"\n\n{col}:")
  print(dataset_df[dataset_df[col]==''].count())



Input:
Input                 0
BBoutput              0
MidasOutput           0
SurfacePlane          0
SurfaceCoordinates    0
dtype: int64


BBoutput:
Input                 74
BBoutput              74
MidasOutput           74
SurfacePlane          74
SurfaceCoordinates    74
dtype: int64


MidasOutput:
Input                 1
BBoutput              1
MidasOutput           1
SurfacePlane          1
SurfaceCoordinates    1
dtype: int64


SurfacePlane:
Input                 62
BBoutput              62
MidasOutput           62
SurfacePlane          62
SurfaceCoordinates    62
dtype: int64


SurfaceCoordinates:
Input                 62
BBoutput              62
MidasOutput           62
SurfacePlane          62
SurfaceCoordinates    62
dtype: int64


In [12]:
non_null_dataset = dataset_df[(dataset_df['BBoutput']!='') & (dataset_df['MidasOutput']!='') & (dataset_df['SurfacePlane']!='') & (dataset_df['SurfaceCoordinates']!='')]

In [13]:
for col in non_null_dataset.columns:
  print(f"\n\n{col}:")
  print(non_null_dataset[non_null_dataset[col]==''].count())



Input:
Input                 0
BBoutput              0
MidasOutput           0
SurfacePlane          0
SurfaceCoordinates    0
dtype: int64


BBoutput:
Input                 0
BBoutput              0
MidasOutput           0
SurfacePlane          0
SurfaceCoordinates    0
dtype: int64


MidasOutput:
Input                 0
BBoutput              0
MidasOutput           0
SurfacePlane          0
SurfaceCoordinates    0
dtype: int64


SurfacePlane:
Input                 0
BBoutput              0
MidasOutput           0
SurfacePlane          0
SurfaceCoordinates    0
dtype: int64


SurfaceCoordinates:
Input                 0
BBoutput              0
MidasOutput           0
SurfacePlane          0
SurfaceCoordinates    0
dtype: int64


In [14]:
dataset_df.shape

(3595, 5)

In [15]:
non_null_dataset.shape

(3459, 5)

In [29]:
non_null_dataset = non_null_dataset.reset_index().iloc[:,1:]

In [30]:
non_null_dataset.to_csv(root+'capstone_metadata.csv')

In [31]:
test_df = pd.read_csv(root+'capstone_metadata.csv')

In [32]:
test_df.shape

(3459, 6)

In [33]:
test_df.head()

,Unnamed: 0,Input,BBoutput,MidasOutput,SurfacePlane,SurfaceCoordinates
0,0,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...
1,1,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...
2,2,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...
3,3,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...
4,4,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...,/content/drive/My Drive/computer_vision/capsto...
